In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from scipy.stats import multivariate_normal

In [2]:
# load data 
datafile = 'dataCh4_7.mat'
data = scipy.io.loadmat(datafile)

# Train 데이터에 대한 정보

In [3]:
# 클래스의 수
K = 3

# 클래스별 표본평균 계산
M = np.vstack([np.mean(data[f'X{i + 1}'], axis=0) for i in range(K)])

# 클래스별 표본공분산 계산
S = np.zeros((2, 2, K))
for i in range(K):
    S[:, :, i] = np.cov(data[f'X{i + 1}'], rowvar=False)

# 클래스별 표본공분산의 평균
smean = np.mean(S, axis=2)

# 학습 데이터 구성
Dtrain = np.vstack([data[f'X{i + 1}'] for i in range(K)])

# 오분류 데이터의 수를 셈
Etrain = np.zeros((3, 1))

# 각 클래스별 데이터의 수
N = data['X1'].shape[0]

# 각 클래스별로 분류 시작
for k in range(1, K + 1):
    X = Dtrain[(k - 1) * N:k * N, :]
    d1 = np.zeros((3, 1))
    d2 = np.zeros((3, 1))
    d3 = np.zeros((3, 1))

    # 각 데이터에 대해 분류 시작
    for i in range(N):
        # 세 개의 판별함수의 값 계산
        # 단위 공분산행렬을 가정한 경우의 판별함수
        d1[:, 0] = np.sum((X[i, :] - M) ** 2, axis=1)
        # 모든 클래스가 동일한 공분산행렬을 가진다고 가정한 경우의 판별함수
        d2[:, 0] = np.sum((X[i, :] - M) @ np.linalg.inv(smean) @ (X[i, :] - M).T, axis=1)
        # 일반적인 공분산행렬을 가정한 경우의 판별함수
        d3[:, 0] = np.sum((X[i, :] - M) @ np.linalg.inv(S[:, :, k - 1]) @ (X[i, :] - M).T, axis=1)

        min1i = np.argmin(d1)  # 각 판별함수 값에 따라 분류
        if min1i != k - 1:
            Etrain[0, 0] += 1
        min2i = np.argmin(d2)
        if min2i != k - 1:
            Etrain[1, 0] += 1
        min3i = np.argmin(d3)
        if min3i != k - 1:
            Etrain[2, 0] += 1

# 오분류율 출력 (학습오차)
Error_rate = Etrain / N
print("Training Error Rate:", Error_rate,sep='\n')

Training Error Rate:
[[0.36]
 [0.4 ]
 [0.3 ]]


# Test 데이터에 대한 정보

In [4]:
# 클래스의 수
K = 3

# 클래스별 표본평균 계산
M = np.vstack([np.mean(data[f'X{i + 1}_test'], axis=0) for i in range(K)])

# 클래스별 표본공분산 계산
S = np.zeros((2, 2, K))
for i in range(K):
    S[:, :, i] = np.cov(data[f'X{i + 1}_test'], rowvar=False)

# 클래스별 표본공분산의 평균
smean = np.mean(S, axis=2)

# 학습 데이터 구성
Dtrain = np.vstack([data[f'X{i + 1}_test'] for i in range(K)])

# 오분류 데이터의 수를 셈
Etrain = np.zeros((3, 1))

# 각 클래스별 데이터의 수
N = data['X1_test'].shape[0]

# 각 클래스별로 분류 시작
for k in range(1, K + 1):
    X = Dtrain[(k - 1) * N:k * N, :]
    d1 = np.zeros((3, 1))
    d2 = np.zeros((3, 1))
    d3 = np.zeros((3, 1))

    # 각 데이터에 대해 분류 시작
    for i in range(N):
        # 세 개의 판별함수의 값 계산
        # 단위 공분산행렬을 가정한 경우의 판별함수
        d1[:, 0] = np.sum((X[i, :] - M) ** 2, axis=1)
        # 모든 클래스가 동일한 공분산행렬을 가진다고 가정한 경우의 판별함수
        d2[:, 0] = np.sum((X[i, :] - M) @ np.linalg.inv(smean) @ (X[i, :] - M).T, axis=1)
        # 일반적인 공분산행렬을 가정한 경우의 판별함수
        d3[:, 0] = np.sum((X[i, :] - M) @ np.linalg.inv(S[:, :, k - 1]) @ (X[i, :] - M).T, axis=1)

        min1i = np.argmin(d1)  # 각 판별함수 값에 따라 분류
        if min1i != k - 1:
            Etrain[0, 0] += 1
        min2i = np.argmin(d2)
        if min2i != k - 1:
            Etrain[1, 0] += 1
        min3i = np.argmin(d3)
        if min3i != k - 1:
            Etrain[2, 0] += 1

# 오분류율 출력 (학습오차)
Error_rate = Etrain / N
print("Test Error Rate:", Error_rate,sep='\n')

Test Error Rate:
[[0.36902]
 [0.36269]
 [0.25136]]
